all relevant hashes in dataview

schedule item loop -> get_item_views(item, beg_week, end_week)

get_item_views updates the relevant dataview hashes including the weekly views for the range of weeks using either caches or creating as needed

create_item_views calls the various handlers such as handle_today

In [208]:
# NOTE: unwrap to encode and wrap to decode @d entries?
import textwrap
import shutil

def wrap(txt, indent=1, width=shutil.get_terminal_size()[0] - 3):
    para = [x.rstrip() for x in txt.split('\n')]
    tmp = []
    first = True
    for p in para:
        if first:
            initial_indent = ''
            first = False
        else:
            initial_indent = ' ' * indent
        tmp.append(
            textwrap.fill(
                p,
                initial_indent=initial_indent,
                subsequent_indent=' ' * indent,
                width=width - indent - 1,
            )
        )
    return '\n'.join(tmp)

def unwrap(wrapped_text):
    # Split the text into paragraphs
    paragraphs = wrapped_text.split('\n')

    # Remove indentations and join lines within each paragraph
    unwrapped_paragraphs = []
    current_paragraph = []

    for line in paragraphs:
        if line.strip() == '':
            # Paragraph separator
            if current_paragraph:
                unwrapped_paragraphs.append(' '.join(current_paragraph))
                current_paragraph = []
            unwrapped_paragraphs.append('')
        else:
            # Remove leading spaces used for indentation
            current_paragraph.append(line.strip())

    # Add the last paragraph if there is any
    if current_paragraph:
        unwrapped_paragraphs.append(' '.join(current_paragraph))

    # Join the unwrapped paragraphs
    return '\n'.join(unwrapped_paragraphs)

# Example usage
original_text = """\
This is a long string that we want to wrap and then unwrap. We will use the textwrap module to handle the wrapping.
This is a new paragraph that should also be wrapped.
"""
original_text += "  Now is the time for all good men to come to the aid of their country.\n" * 5

# Wrap the text
wrapped_text = wrap(original_text, indent=3, width=50)
print("Wrapped text:")
print(wrapped_text)

# Unwrap the text
unwrapped_text = unwrap(wrapped_text)
print("\nUnwrapped text:")
print(unwrapped_text)

Wrapped text:
This is a long string that we want to wrap and
   then unwrap. We will use the textwrap
   module to handle the wrapping.
   This is a new paragraph that should also be
   wrapped.
     Now is the time for all good men to come
   to the aid of their country.
     Now is the time for all good men to come
   to the aid of their country.
     Now is the time for all good men to come
   to the aid of their country.
     Now is the time for all good men to come
   to the aid of their country.
     Now is the time for all good men to come
   to the aid of their country.


Unwrapped text:
This is a long string that we want to wrap and then unwrap. We will use the textwrap module to handle the wrapping. This is a new paragraph that should also be wrapped. Now is the time for all good men to come to the aid of their country. Now is the time for all good men to come to the aid of their country. Now is the time for all good men to come to the aid of their country. Now is the time fo

In [4]:
from dateutil.rrule import rrule, rruleset, rrulestr, DAILY

ruleset = rrulestr("""
DTSTART:19970902T090000
RRULE:FREQ=DAILY;INTERVAL=10;COUNT=5
RRULE:FREQ=DAILY;INTERVAL=5;COUNT=3
""")

list(ruleset)



[datetime.datetime(1997, 9, 2, 9, 0),
 datetime.datetime(1997, 9, 7, 9, 0),
 datetime.datetime(1997, 9, 12, 9, 0),
 datetime.datetime(1997, 9, 22, 9, 0),
 datetime.datetime(1997, 10, 2, 9, 0),
 datetime.datetime(1997, 10, 12, 9, 0)]

In [23]:
import re
wkd_regex = re.compile(r'(?<![\w-])([+-][1-4])?(MO|TU|WE|TH|FR|SA|SU)(?!\w)')
# wkd_regex = re.compile(r'([+-][1-4])?(MO|TU|WE|TH|FR|SA|SU)')
wkd_str = ("-1MO, +2Tu, WE, -5TH, 1FR, -3SA, 5SU").upper()
matches = wkd_regex.findall(wkd_str)
_ = [f"{x[0]}{x[1]}" for x in matches]
all = [x.strip() for x in wkd_str.split(',')]
bad = [x for x in all if x not in good]
good = _
# for x in matches:
#     arg = f"+{x[0]}" if x[0] and not x[0].startswith('-') else x[0]
#     s = f"{x[0]{x[1]}({arg})" if arg else f"rrule.{x[1]}"
#     good.append(eval(s))

print('\nall:', all, '\nbad:', bad, '\ngood:', good)


all: ['-1MO', '+2TU', 'WE', '-5TH', '1FR', '-3SA', '5SU'] 
bad: ['-5TH', '1FR', '5SU'] 
good: ['-1MO', '+2TU', 'WE', '-3SA']


## Views

- iterate through all items appending relevant 

- agenda (cached?). Create collects agenda rows from all items, sorts and makes tree

# TODO

-  _from_entry_string: for Item, Repeat and Jobs. 

- Clean up Dataview and Item classes - what's actually needed for schedule?

- 

## Unit Tests

In [168]:
import re
from dateutil.rrule import rruleset, rrule, DAILY
from datetime import datetime, timedelta
import pytz

class Item:
    def __init__(self, json_dict=None, input_string=None):
        self.created = self._get_current_timestamp()
        self.itemtype = None
        self.summary = None
        self.start = None
        self.end = None
        self.recurrence = None
        self.rruleset = None
        if json_dict:
            self._init_from_json(json_dict)
        elif input_string:
            self.parse_input(input_string)

    def _get_current_timestamp(self):
        return datetime.now(pytz.utc).strftime("%Y%m%dT%H%M%S")

    def _init_from_json(self, json_dict):
        self.created = json_dict.get("created", self.created)
        self.itemtype = json_dict.get("itemtype")
        self.summary = json_dict.get("summary")
        self.start = self._parse_datetime(json_dict.get("s", "").replace("{T}:", ""))
        self.end = self._parse_datetime(json_dict.get("e", "").replace("{T}:", ""))
        self.recurrence = json_dict.get("r", [{}])[0]

    def parse_input(self, input_string):
        tokens = self._tokenize(input_string)
        self._parse_tokens(tokens)
        self._validate()

    def _tokenize(self, input_string):
        pattern = r'(@\w+ [^@&]+)|(&\w+ \S+)|(^\S+)|(\S[^@&]*)'
        matches = re.findall(pattern, input_string)
        return [match[0] or match[1] or match[2] or match[3] for match in matches if match[0] or match[1] or match[2] or match[3]]

    def _parse_tokens(self, tokens):
        self.itemtype = tokens[0][0]
        summary_tokens = []
        recurrence_attributes = {}
        for token in tokens[1:]:
            if token.startswith('@'):
                break
            summary_tokens.append(token)
        self.summary = ' '.join(summary_tokens)
        for token in tokens[len(summary_tokens) + 1:]:
            if token.startswith('@s'):
                self.start = self._parse_datetime(token[3:].strip())
            elif token.startswith('@e'):
                self.end = self._parse_duration(token[3:].strip())
            elif token.startswith('@r'):
                self.recurrence = self._parse_recurrence(token[3:].strip())
            elif token.startswith('&'):
                attribute, value = self._parse_attribute(token)
                recurrence_attributes[attribute] = value
        if self.recurrence:
            self.recurrence.update(recurrence_attributes)

    def _parse_datetime(self, datetime_str):
        if not datetime_str:
            return None
        try:
            return datetime.strptime(datetime_str, "%Y%m%dT%H%M%S")
        except ValueError:
            try:
                return datetime.strptime(datetime_str, "%Y/%m/%d")
            except ValueError:
                raise ValueError(f"Invalid datetime format: {datetime_str}")

    def _parse_duration(self, duration_str):
        match = re.match(r'(\d+)([dwmy])', duration_str)
        if not match:
            raise ValueError(f"Invalid duration format: {duration_str}")
        value, unit = match.groups()
        if unit == 'd':
            return timedelta(days=int(value))
        elif unit == 'w':
            return timedelta(weeks=int(value))
        elif unit == 'm':
            return timedelta(days=int(value) * 30)
        elif unit == 'y':
            return timedelta(days=int(value) * 365)

    def _parse_recurrence(self, recurrence_str):
        return {"r": recurrence_str}

    def _parse_attribute(self, attribute_str):
        key, value = attribute_str[1:].split()
        if key == "M":
            value = [int(value)]
        elif key == "w":
            value = [f"{value[:1]}TH"]
        return key, value

    def _validate(self):
        if self.itemtype == '*' and not self.start:
            raise ValueError("Events must have a start datetime (@s)")
        if self.recurrence and not self.start:
            raise ValueError("Items with recurrence (@r) must have a start datetime (@s)")

    def to_dict(self):
        data = {
            "created": self.created,
            "itemtype": self.itemtype,
            "summary": self.summary,
        }
        if self.start:
            data["s"] = "{T}:" + self.start.strftime("%Y%m%dT%H%M%S")
        if self.end:
            data["e"] = "{T}:" + (self.start + self.end).strftime("%Y%m%dT%H%M%S")
        if self.recurrence:
            data["r"] = [self.recurrence]
        return data

    def __repr__(self):
        return str(self.to_dict())

# Example usage
json_entry = {
    "created": "{T}:20240712T1052",
    "itemtype": "*",
    "summary": "Thanksgiving",
    "s": "{T}:20101126T0500",
    "r": [
        {
            "r": "y",
            "M": [11],
            "w": ["4TH"]
        }
    ],
    "modified": "{T}:20240712T1054"
}

item_from_json = Item(json_dict=json_entry)
print(item_from_json)

item_from_string = Item(input_string="* Thanksgiving @s 2010/11/26 @r y &M 11 &w 4TH")
print(item_from_string)

{'created': '{T}:20240712T1052', 'itemtype': '*', 'summary': 'Thanksgiving', 's': '{T}:20101126T050000', 'r': [{'r': 'y', 'M': [11], 'w': ['4TH']}]}
{'created': '20240722T124522', 'itemtype': '*', 'summary': 'Thanksgiving ', 's': '{T}:20101126T000000', 'r': [{'r': 'y', 'M': [11], 'w': ['4TH']}]}


In [ ]:
import re
from dateutil.rrule import rruleset, rrule, DAILY
from datetime import datetime, timedelta
import pytz

class Item:
    def __init__(self, input_string=None):
        self.created = self._get_current_timestamp()
        self.itemtype = None
        self.summary = None
        self.start = None
        self.end = None
        self.recurrence = None
        self.rruleset = None
        if input_string:
            self.parse_input(input_string)

    def _get_current_timestamp(self):
        return datetime.now(pytz.utc).strftime("%Y%m%dT%H%M%S")

    def parse_input(self, input_string):
        tokens = self._tokenize(input_string)
        self._parse_tokens(tokens)
        self._validate()

    def _tokenize(self, input_string):
        pattern = r'(@\w+ [^@]+)|(^\S+)|(\S[^@]*)'
        matches = re.findall(pattern, input_string)
        return [match[0] or match[1] or match[2] for match in matches if match[0] or match[1] or match[2]]

    def _parse_tokens(self, tokens):
        self.itemtype = tokens[0][0]
        summary_tokens = []
        for token in tokens[1:]:
            if token.startswith('@'):
                break
            summary_tokens.append(token)
        self.summary = ' '.join(summary_tokens)
        for token in tokens[len(summary_tokens) + 1:]:
            if token.startswith('@s'):
                self.start = self._parse_datetime(token[3:].strip())
            elif token.startswith('@e'):
                self.end = self._parse_duration(token[3:].strip())
            elif token.startswith('@r'):
                self.recurrence = self._parse_recurrence(token[3:].strip())
            # Add additional parsing as needed

    def _parse_datetime(self, datetime_str):
        try:
            return datetime.strptime(datetime_str, "%Y/%m/%d")
        except ValueError:
            raise ValueError(f"Invalid datetime format: {datetime_str}")

    def _parse_duration(self, duration_str):
        match = re.match(r'(\d+)([dwmy])', duration_str)
        if not match:
            raise ValueError(f"Invalid duration format: {duration_str}")
        value, unit = match.groups()
        if unit == 'd':
            return timedelta(days=int(value))
        elif unit == 'w':
            return timedelta(weeks=int(value))
        elif unit == 'm':
            return timedelta(days=int(value) * 30)
        elif unit == 'y':
            return timedelta(days=int(value) * 365)

    def _parse_recurrence(self, recurrence_str):
        # Implement recurrence parsing logic
        pass

    def _validate(self):
        if self.itemtype == '*' and not self.start:
            raise ValueError("Events must have a start datetime (@s)")
        if self.recurrence and not self.start:
            raise ValueError("Items with recurrence (@r) must have a start datetime (@s)")

    def to_dict(self):
        data = {
            "created": self.created,
            "itemtype": self.itemtype,
            "summary": self.summary,
        }
        if self.start:
            data["s"] = self.start.strftime("%Y%m%dT%H%M%S")
        if self.end:
            data["e"] = (self.start + self.end).strftime("%Y%m%dT%H%M%S")
        if self.recurrence:
            data["r"] = self.recurrence
        return data

    def __repr__(self):
        return str(self.to_dict())

# Example usage
item = Item("* carpe diem @s 2024/7/10 @r d")
print(item)

item2 = Item("- ask ChatGPT how to fix my code")
print(item2)

In [ ]:
class Date:
    def __init__(self, year, month, day):
        self.year = year
        self.month = month
        self.day = day

    @classmethod
    def from_string(cls, date_str):
        year, month, day = map(int, date_str.split('-'))
        return cls(year, month, day)

date1 = Date(2024, 7, 5)
date2 = Date.from_string("2024-07-05")

In [ ]:
class Calculator:
    constants = {
        "pi": 3.14,
        "e": 2.71}

    @classmethod
    def add(cls, a, b):
        return cls.constants.get(a, a) + cls.constants.get(b, b)

    @classmethod
    def subtract(cls, a, b):
        return cls.constants.get(a, a) - cls.constants.get(b, b)

print(Calculator.add(5, 3))
print(Calculator.subtract(5, 3))
print(Calculator.add(5, "pi"))

In [4]:
# FIXME: not working
from datetime import datetime, date, timedelta
from dateutil import rrule
from dateutil.rrule import rruleset, DAILY, rrulestr
from dateutil.tz import gettz
import textwrap

print(f"{dict(a='three', b='two')}")

# def is_naive(dt):
#     return dt.tzinfo is None or dt.tzinfo.utcoffset(dt) is None

# Function to create a string representation of the rruleset
def rruleset_to_string(rruleset_obj):
    parts = []
    # parts.append("rrules:")
    for rule in rruleset_obj._rrule:
        # parts.append(f"{textwrap.fill(str(rule))}")
        parts.append(f"{'\\n'.join(str(rule).split('\n'))}")
    # parts.append("exdates:")
    for exdate in rruleset_obj._exdate:
        parts.append(f"EXDATE:{exdate}")
    # parts.append("rdates:")
    for rdate in rruleset_obj._rdate:
        parts.append(f"RDATE:{rdate}")
    return "\n".join(parts)

# Define the timezone (replace 'America/New_York' with your specific timezone)
pacific = gettz('US/Pacific')
mountain = gettz('America/Denver')
central = gettz('US/Central')
eastern = gettz('America/New_York')
local = gettz()
utc = gettz('UTC')
naive = None

tz = naive
# Define the start date
start_date = datetime(2024,7,20,13,0,0).astimezone().replace(tzinfo=None)


def dt_to_naive(dt):
    return dt if dt.tzinfo is None or dt.tzinfo.utcoffset(dt) is None else dt.replace(tzinfo=None)

no_r = rruleset()
no_r.rdate(start_date)
no_r.rdate(start_date + timedelta(days=5))
print(f"no_r = {rruleset_to_string(no_r)}")
print(f"no_r dates: {list(no_r)}")

rules_lst = []

# Create a recurrence rule for daily events
rule1 = rrule.rrule(freq=DAILY, dtstart=start_date, count=14)
rules_lst.append(str(rule1))
# Create another recurrence rule for specific days (e.g., every 2 days)
rule2 = rrule.rrule(freq=DAILY, interval=2, count=7)
rules_lst.append(str(rule2))
print(f"{rules_lst = }")

# Create an rruleset
rules1 = rruleset()
rules2 = rruleset()
rules12 = rruleset()

# Add the rules to the rruleset
rules1.rrule(rule1)
rules2.rrule(rule2)
rules12.rrule(rule1)
rules12.rrule(rule2)

# Add a specific date to include
plusdates = [datetime(2024, 11, 4, 13, 45, tzinfo=tz), datetime(2024, 11, 5, 15, 15, tzinfo=tz)]
for dt in plusdates:
    dt = dt_to_naive(dt)
    rules1.rdate(dt)  # dt_to_naive(dt)
    rules2.rdate(dt)  # dt_to_naive(dt)
    rules12.rdate(dt)  # dt_to_naive(dt)
    rules_lst.append(dt.strftime("RDATE:%Y%m%dT%H%M%S"))

# Add a specific date to exclude
minusdates = [datetime(2024, 11, 4, 13, 30, tzinfo=tz),]
for dt in minusdates:
    dt = dt_to_naive(dt)
    rules12.exdate(dt)
    rules_lst.append(dt.strftime("EXDATE:%Y%m%dT%H%M%S"))

# Generate the occurrences of the event
occurrences1 = list(rules1)
occurrences2 = list(rules2)
occurrences12 = list(rules12)

# xafter_eg = rules12.xafter(start_date, count=3, inc=True)
# print(f"xafter_eg = {list(after_eg) = }")

# between_eg = rules12.between(datetime.now(), datetime.now() + timedelta(days=14), count=3, inc=True)
# print(f"between_eg = {list(between_eg) = }")

print(f"\nrules:")
print(rruleset_to_string(rules12))
print(f"{rules12.__dict__ = }")

# Print the occurrences
print("\noccurrences from rules1:")
for occurrence in occurrences1:
    print(occurrence.strftime("  %a %Y-%m-%d %H:%M %Z %z"))

print("\noccurrences from rules2:")
for occurrence in occurrences2:
    print(occurrence.strftime("  %a %Y-%m-%d %H:%M %Z %z"))

print("\noccurrences from rules12:")
for occurrence in occurrences12:
    print(occurrence.strftime("  %a %Y-%m-%d %H:%M %Z %z"))

# print("\nlist of string representations of rules:")
# print('\n'.join(rules_lst))

# print("\nfrom list of string representations to new rules:")
# rules_from_str = rrulestr('\n'.join(rules_lst))
# print(rruleset_to_string(rules_from_str))

# occurrences_from_str = list(rules_from_str)
# print("\noccurrences from new rules:")
# for occurrence in occurrences_from_str:
#     print(occurrence.strftime("  %a %Y-%m-%d %H:%M %Z %z"))

{'a': 'three', 'b': 'two'}
no_r = RDATE:2024-07-20 13:00:00
RDATE:2024-07-25 13:00:00
no_r dates: [datetime.datetime(2024, 7, 20, 13, 0), datetime.datetime(2024, 7, 25, 13, 0)]
rules_lst = ['DTSTART:20240720T130000\nRRULE:FREQ=DAILY;COUNT=14', 'DTSTART:20240723T095034\nRRULE:FREQ=DAILY;INTERVAL=2;COUNT=7']

rules:
DTSTART:20240720T130000\nRRULE:FREQ=DAILY;COUNT=14
DTSTART:20240723T095034\nRRULE:FREQ=DAILY;INTERVAL=2;COUNT=7
EXDATE:2024-11-04 13:30:00
RDATE:2024-11-04 13:45:00
RDATE:2024-11-05 15:15:00
rules12.__dict__ = {'_cache': None, '_cache_complete': False, '_len': 23, '_rrule': [<dateutil.rrule.rrule object at 0x107133b60>, <dateutil.rrule.rrule object at 0x1108c8b00>], '_rdate': [datetime.datetime(2024, 11, 4, 13, 45), datetime.datetime(2024, 11, 5, 15, 15)], '_exrule': [], '_exdate': [datetime.datetime(2024, 11, 4, 13, 30)]}

occurrences from rules1:
  Sat 2024-07-20 13:00  
  Sun 2024-07-21 13:00  
  Mon 2024-07-22 13:00  
  Tue 2024-07-23 13:00  
  Wed 2024-07-24 13:00  
  Th

In [170]:
import re

# Define the regex pattern
pattern = re.compile(r'(?<![\w-])(-?[1-4]?)(MO|TU|WE|TH|FR|SA|SU)(?!\w)')

# Example test string
test_string = "MO, -1TU, 4FR, WE, SA, -3MO, 2WE, -4FR, 5FR, XYZ, -5MO"

# Find all matches
matches = re.findall(pattern, test_string)

def maybe_int(x):
    try:
        return int(x)
    except ValueError:
        return x

good = [f"{maybe_int(x[0])}{x[1]}" for x in matches]
all = [x.strip() for x in test_string.split(',')]
bad = [x for x in all if x not in good]

print("good:", good)
print("all:", all)
print("bad:", bad)

# Print the matches
print(matches)

good: ['MO', '-1TU', '4FR', 'WE', 'SA', '-3MO', '2WE', '-4FR']
all: ['MO', '-1TU', '4FR', 'WE', 'SA', '-3MO', '2WE', '-4FR', '5FR', 'XYZ', '-5MO']
bad: ['5FR', 'XYZ', '-5MO']
[('', 'MO'), ('-1', 'TU'), ('4', 'FR'), ('', 'WE'), ('', 'SA'), ('-3', 'MO'), ('2', 'WE'), ('-4', 'FR')]


# NOTE: Tracking
## Tracking

For undated tasks, add option for adding @o t to add completions to @h instead of simply marking the task as completed by adding an @f entry.

One possibility would be to have @h be a tuple of (number of completions, average timedelta between completions, datetime of last completion)

In [14]:
# NOTE: Tracker and TrackerManager Classes
from typing import List, Dict, Any, Callable, Mapping
from collections import defaultdict
from datetime import datetime, date, timedelta
from dateutil.parser import parse
import traceback
import sys

default_history = [0, timedelta(minutes=0), None]
ZERO = timedelta(minutes=0)

class Tracker:
    _next_id = 1
    default_history = [0, timedelta(minutes=0), None]

    @classmethod
    def format_dt(self, dt: Any) -> str:
        if not isinstance(dt, datetime):
            return ""
        return dt.strftime("%Y-%m-%d %H:%M")

    @classmethod
    def parse_dt(self, dt: str = "") -> datetime:
        # print(f"parsing {dt = }; type(dt) = {type(dt)}")
        if isinstance(dt, datetime):
            return dt
        elif isinstance(dt, str) and dt:
            try:
                # print(f"parsing {dt = }")
                dt = parse(dt)
                # print(f"returning {dt = }")
                return dt
            except Exception as e:
                print(f"Error parsing datetime: {dt}\ne {repr(e)}\n{traceback.format_exc()}", file=sys.stderr, flush=True)
                return None
        else:
            return None

    def __init__(self, name: str, dt: str = "") -> None:
        self.doc_id = Tracker._next_id
        Tracker._next_id += 1
        self.name = name
        self.last_completion = None
        self.last_interval = None
        self.next_expected_completion = None
        self.history = Tracker.default_history

    def record_completion(self, dt):
        self.last_completion = Tracker.parse_dt(dt)
        # print(f"{dt = } -> {self.last_completion = }")
        # print(f"self.last_completion: {Tracker.format_dt(self.last_completion)}")

        num_intervals, average_interval, last_completion = self.history
        if last_completion is None:
            num_intervals = 0
            average_interval = ZERO
            last_completion = self.last_completion
        else:
            self.last_interval = self.last_completion - last_completion
            total = num_intervals * average_interval + self.last_interval
            num_intervals += 1
            average_interval = total / num_intervals

        self.history = [num_intervals, average_interval, self.last_completion]
        self
        self.next_expected_completion = self.last_completion + average_interval if (self.last_completion is not None and self.history[1] is not ZERO) else None

    def get_tracker_data(self):
        # print(f"{self.history = }")
        return f"""{self.name}:
    completion intervals: {self.history[0]}
    average interval: {str(self.history[1])}
    last interval: {str(self.last_interval)}
    last_completion: {Tracker.format_dt(self.last_completion)}
    next_expected_completion: {Tracker.format_dt(self.next_expected_completion)}"""

class TrackerManager:
    def __init__(self) -> None:
        self.trackers = {}

    def add_tracker(self, tracker) -> None:
        doc_id = tracker.doc_id
        self.trackers[doc_id] = tracker

    def record_completion(self, doc_id: int, dt: datetime):
        self.trackers[doc_id].record_completion(dt)
        print(f"""\
{doc_id}: Recorded {dt.strftime('%Y-%m-%d %H:%M')} as a completion  {self.trackers[doc_id].get_tracker_data()}""")

    def get_tracker_data(self, doc_id: int = None):
        if doc_id is None:
            for k, v in self.trackers.items():
                print(f"{k}: {v.get_tracker_data()}")
        elif doc_id in self.trackers:
            print(f"{doc_id}: {self.trackers[doc_id].get_tracker_data()}")

    def list_trackers(self):
        for k, v in self.trackers.items():
            print(f"{k}: {v.name}")

    def __repr__(self) -> str:
        for key, tracker in self.trackers.items():
            print(f"{key}: {tracker.get_tracker_data()}")

print("creating manager")
manager = TrackerManager()
print("adding tracker for bird feeder")
manager.add_tracker(Tracker("fill bird feeders"))
datetimes = [parse("2024/6/23 5:38p"),  parse("2024/7/6 4:44p"), parse("2024/7/21 4:18p"), parse("2024/7/26 3pm"), parse("2024/8/2/4p")]
for dt in datetimes:
    manager.record_completion(1, dt)

for name in ['fill water dispenser', 'haircut', 'fill cat feeder']:
    manager.add_tracker(Tracker(name))

manager.record_completion(2, parse("6:35p"))

datetimes = [datetime(2024, 7, 20, 13, 45), datetime(2024, 7, 22, 15, 15), datetime(2024, 7, 25, 9, 45), datetime(2024, 7, 26, 18, 15)]
print("adding tracker for test")
manager.add_tracker(Tracker("test tracker"))
for dt in datetimes:
    manager.record_completion(5, dt)

print("\nall manager data:")
manager.get_tracker_data()
manager.list_trackers()
print(f"{manager.__repr__() = }")

creating manager
adding tracker for bird feeder
1: Recorded 2024-06-23 17:38 as a completion  fill bird feeders:
    completion intervals: 0
    average interval: 0:00:00
    last interval: None
    last_completion: 2024-06-23 17:38
    next_expected_completion: 
1: Recorded 2024-07-06 16:44 as a completion  fill bird feeders:
    completion intervals: 1
    average interval: 12 days, 23:06:00
    last interval: 12 days, 23:06:00
    last_completion: 2024-07-06 16:44
    next_expected_completion: 2024-07-19 15:50
1: Recorded 2024-07-21 16:18 as a completion  fill bird feeders:
    completion intervals: 2
    average interval: 13 days, 23:20:00
    last interval: 14 days, 23:34:00
    last_completion: 2024-07-21 16:18
    next_expected_completion: 2024-08-04 15:38
1: Recorded 2024-07-26 15:00 as a completion  fill bird feeders:
    completion intervals: 3
    average interval: 10 days, 23:07:20
    last interval: 4 days, 22:42:00
    last_completion: 2024-07-26 15:00
    next_expected_c

In [177]:
import re

def get_int_and_str(s):
    # Define a regular expression to match an optional sign followed by digits
    weekdays = ['MO', 'TU', 'WE', 'TH', 'FR', 'SA', 'SU']
    weekday_str = ','.join(weekdays)
    match = re.match(r'^([+-]?\d*)(.*)$', s)
    if match:
        integer_part = match.group(1)
        string_part = match.group(2)
        # Convert integer_part to an integer if it's not empty, otherwise None
        integer_part = integer_part if integer_part else None
        # problems = []
        # if string_part not in weekdays:
        #     problems.append(f"'{string_part}' is not a valid weekday from  {weekday_str}")


        return integer_part, string_part
    return None, s  # Default case if no match is found

# Example usage
examples = ['1MO', '-2T', 'S', '+1FD', '-3WD', '']

for example in examples:
    integer_part, string_part = get_int_and_str(example)
    print(f"Original: {example} -> Integer part: {integer_part} (type: {type(integer_part)}), String part: {string_part}")

Original: 1MO -> Integer part: 1 (type: <class 'str'>), String part: MO
Original: -2T -> Integer part: -2 (type: <class 'str'>), String part: T
Original: S -> Integer part: None (type: <class 'NoneType'>), String part: S
Original: +1FD -> Integer part: +1 (type: <class 'str'>), String part: FD
Original: -3WD -> Integer part: -3 (type: <class 'str'>), String part: WD
Original:  -> Integer part: None (type: <class 'NoneType'>), String part: 


In [3]:
import re
from math import ceil
from datetime import datetime
from dateutil.rrule import rruleset
from dateutil.parser import parse

class Item:
    token_keys = {
        'itemtype': [
            'item type',
            'character from * (event), - (task), % (journal), ~ (goal), + (track) or ! (inbox)',
            'do_itemtype',
        ],
        'summary': [
            'summary',
            "brief item description. Append an '@' to add an option.",
            'do_summary',
        ],
        's': ['scheduled', 'starting date or datetime', 'do_datetime'],
        'r': ['recurrence', 'recurrence rule', 'do_rrule'],
        'j': ['journal', 'journal entry', 'do_j'],
        '+': ['rdate', 'recurrence dates', 'do_rdate'],
        '-': ['exdate', 'exception dates', 'do_exdate'],
        'rw': ['weekdays', 'list from SU, MO, ..., SA, possibly prepended with a positive or negative integer', 'do_weekdays'],
        'rM': ['months', 'list of integers in 1 ... 12', 'do_months'],
        # Add more `&` token handlers for @j here as needed
    }

    wkd_list = ['SU', 'MO', 'TU', 'WE', 'TH', 'FR', 'SA']
    wkd_str = ', '.join(wkd_list)

    def __init__(self):
        self.entry = ""
        self.tokens = []
        self.previous_entry = ""
        self.parsed_data = {}
        self.previous_tokens = []
        self.rrule_tokens = []
        self.job_tokens = []
        self.rrules = []
        self.rdates = []
        self.exdates = []
        self.dtstart = None

    def parse_input(self, entry: str):
        """
        Parses the input string to extract tokens, then processes and validates the tokens.
        """
        digits = '1234567890' * ceil(len(entry) / 10)
        self._tokenize(entry)
        print(f'entry to tokens:\n   |{digits[:len(entry)]}|\n   |{entry}|\n   {self.tokens}')
        self._parse_tokens(entry)
        self.previous_entry = entry
        self.previous_tokens = self.tokens.copy()

    def _tokenize(self, entry: str):
        self.entry = entry
        pattern = r'(@\w+ [^@]+)|(^\S+)|(\S[^@]*)'
        matches = re.finditer(pattern, self.entry)
        tokens_with_positions = []
        for match in matches:
            # Get the matched token
            token = match.group(0)
            # Get the start and end positions
            start_pos = match.start()
            end_pos = match.end()
            # Append the token and its positions as a tuple
            tokens_with_positions.append((token, start_pos, end_pos))
        self.tokens = tokens_with_positions

    def _parse_tokens(self, entry: str):
        if not self.previous_entry:
            # If there is no previous entry, parse all tokens
            self._parse_all_tokens()
            return

        # Identify the affected tokens based on the change
        changes = self._find_changes(self.previous_entry, entry)
        affected_tokens = self._identify_affected_tokens(changes)

        # Parse only the affected tokens
        for token_info in affected_tokens:
            token, start_pos, end_pos = token_info
            # Check if the token has actually changed
            if self._token_has_changed(token_info):
                print(f"processing changed token: {token_info}")
                if start_pos == 0:
                    self._dispatch_token(token, start_pos, end_pos, 'itemtype')
                elif start_pos == 2:
                    self._dispatch_token(token, start_pos, end_pos, 'summary')
                else:
                    self._dispatch_token(token, start_pos, end_pos)

    def _parse_all_tokens(self):
        for i, token_info in enumerate(self.tokens):
            token, start_pos, end_pos = token_info
            if i == 0:
                self._dispatch_token(token, start_pos, end_pos, 'itemtype')
            elif i == 1:
                self._dispatch_token(token, start_pos, end_pos, 'summary')
            else:
                token_type = token.split()[0][1:]  # Extract token type (e.g., 's' from '@s')
                self._dispatch_token(token, start_pos, end_pos, token_type)

    def _find_changes(self, previous: str, current: str):
        # Find the range of changes between the previous and current strings
        start = 0
        while start < len(previous) and start < len(current) and previous[start] == current[start]:
            start += 1

        end_prev = len(previous)
        end_curr = len(current)

        while end_prev > start and end_curr > start and previous[end_prev - 1] == current[end_curr - 1]:
            end_prev -= 1
            end_curr -= 1

        return start, end_curr

    def _identify_affected_tokens(self, changes):
        start, end = changes
        affected_tokens = []
        for token_info in self.tokens:
            token, start_pos, end_pos = token_info
            if start <= end_pos and end >= start_pos:
                affected_tokens.append(token_info)
        return affected_tokens

    def _token_has_changed(self, token_info):
        return token_info not in self.previous_tokens

    def _dispatch_token(self, token, start_pos, end_pos, token_type=None):
        if token_type is None:
            if token.startswith('@'):
                token_type = token.split()[0][1:]  # Extract token type (e.g., 's' from '@s')
            else:
                token_type = token
        if token_type in self.token_keys:
            print(f"Dispatching token: {token} as {token_type}")
            method_name = self.token_keys[token_type][2]
            method = getattr(self, method_name)
            is_valid, result, sub_tokens = method(token)
            if is_valid:
                if token_type == 'r':
                    print(f"appending {result} to self.rrules")
                    self.rrules.append(result)
                    self._dispatch_sub_tokens(sub_tokens, 'r')
                elif token_type == 'j':
                    self.job_tokens.append(result)
                    self._dispatch_sub_tokens(sub_tokens, 'j')
                elif token_type == '+':
                    self.rdates.extend(result)
                elif token_type == '-':
                    self.exdates.extend(result)
                else:
                    self.parsed_data[token_type] = result
            else:
                print(f"Error processing token '{token_type}': {result}")
        else:
            print(f"No handler for token: {token}")

    def _dispatch_sub_tokens(self, sub_tokens, prefix):
        for part in sub_tokens:
            if part.startswith('&'):
                token_type = prefix + part[1:2]  # Prepend prefix to token type
                token_value = part[2:].strip()
                print(f"token_type = '{token_type}': token_value = '{token_value}'")
                if token_type in self.token_keys:
                    method_name = self.token_keys[token_type][2]
                    method = getattr(self, method_name)
                    is_valid, result = method(token_value)
                    print(f"{token_value} => {result}")
                    if is_valid:
                        self.rrule_tokens[-1][1][token_type] = result
                    else:
                        print(f"Error processing sub-token '{token_type}': {result}")
                else:
                    print(f"No handler for sub-token: {token_type}")

    def _validate(self):
        # Overall validation logic if needed
        pass

    @classmethod
    def do_itemtype(cls, token):
        # Process item type token
        print(f"Processing item type token: {token}")
        valid_itemtypes = {'*', '-', '%', '~', '+', '!'}
        itemtype = token[0]
        if itemtype in valid_itemtypes:
            return True, itemtype, []
        else:
            return False, f"Invalid item type: {itemtype}", []

    @classmethod
    def do_summary(cls, token):
        # Process summary token
        print(f"Processing summary token: {token}")
        if len(token) >= 1:
            return True, token.strip(), []
        else:
            return False, "Summary cannot be empty", []

    def do_datetime(self, token):
        # Process datetime token
        print(f"Processing datetime token: {token}")
        try:
            datetime_str = token.split()[1]
            datetime_obj = parse(datetime_str)
            self.dtstart = datetime_obj
            return True, datetime_obj, []
        except ValueError as e:
            return False, f"Invalid datetime: {datetime_str}. Error: {e}", []

    def do_rrule(self, token):
        # Process rrule token
        print(f"Processing rrule token: {token}")
        freq_map = dict(
            y='YEARLY', m='MONTHLY', w='WEEKLY', d='DAILY', h='HOURLY', n='MINUTELY')
        parts = token.split()
        print(f"do_rrule {parts = }")
        if len(parts) < 2:
            return False, f"Missing rrule frequency: {token}", []
        elif parts[1] not in freq_map:
            keys = ", ".join([f"{k}: {v}" for k, v in freq_map.items()])
            return False, f"'{parts[1]}', is not one of the supported frequencies from: \n   {keys}", []
        freq = freq_map[parts[1]]
        rrule_params = {'FREQ': freq}
        if self.dtstart:
            rrule_params['DTSTART'] = self.dtstart.strftime('%Y%m%dT%H%M%S')

        # Collect & tokens that follow @r
        sub_tokens = self._extract_sub_tokens(token, '&')

        self.rrule_tokens.append((token, rrule_params))
        print(f"{self.rrule_tokens = }")
        return True, rrule_params, sub_tokens

    def do_job(self, token):
        # Process journal token
        print(f"Processing journal token: {token}")
        journal_params = {}

        # Collect & tokens that follow @j
        sub_tokens = self._extract_sub_tokens(token, '&')

        self.job_tokens.append((token, journal_params))
        return True, journal_params, sub_tokens

    def _extract_sub_tokens(self, token, delimiter):
        # Use regex to extract sub-tokens
        pattern = rf'({delimiter}\w+ \S+)'
        matches = re.findall(pattern, token)
        return matches

    @classmethod
    def do_weekdays(cls, wkd_str: str):
        """
        Converts a string representation of weekdays into a list of rrule objects.
        """
        wkd_str = wkd_str.upper()
        wkd_regex = r'(?<![\w-])([+-][1-4])?(MO|TU|WE|TH|FR|SA|SU)(?!\w)'
        print(f"in do_weekdays with wkd_str = |{wkd_str}|")
        matches = re.findall(wkd_regex, wkd_str)
        _ = [f"{x[0]}{x[1]}" for x in matches]
        all = [x.strip() for x in wkd_str.split(',')]
        bad = [x for x in all if x not in _]
        problem_str = ""
        problems = []
        print(f"{all = }, {bad = }")
        for x in bad:
            probs = []
            print(f"splitting {x}")
            i, w = cls.split_int_str(x)
            print(f"{x = }, i = |{i}|, w = |{w}|")
            if i is not None:
                abs_i = abs(int(i))
                if abs_i > 4 or abs_i == 0:
                    probs.append(f"{i} must be between -4 and -1 or between +1 and +4")
                elif not (i.startswith('+') or i.startswith('-')):
                    probs.append(f"{i} must begin with '+' or '-'")
            w = w.strip()
            if not w:
                probs.append(f"Missing weekday abbreviation from {cls.wkd_str}")
            elif w not in cls.wkd_list:
                probs.append(f"{w} must be a weekday abbreviation from {cls.wkd_str}")
            if probs:
                problems.append(f"In '{x}': {', '.join(probs)}")
            else:
                # undiagnosed problem
                problems.append(f"{x} is invalid")
        if problems:
            problem_str = f"Problem entries: {', '.join(bad)}\n{'\n'.join(problems)}"
        good = []
        for x in matches:
            s = f"{x[0]}{x[1]}" if x[0] else f"{x[1]}"
            good.append(s)
        good_str = ','.join(good)
        if problem_str:
            return False, f"{problem_str}\n{good_str}"
        else:
            return True, f"BYDAY={good_str}"

    @classmethod
    def do_months(cls, arg):
        """
        Process a comma separated list of integer month numbers from 1, 2, ..., 12
        """
        monthsstr = 'months: a comma separated list of integer month numbers from 1, 2, ..., 12'
        if arg:
            args = arg.split(',')
            ok, res = cls.integer_list(args, 0, 12, False, '')
            if ok:
                obj = res
                rep = f'{arg}'
            else:
                obj = None
                rep = f'invalid months: {res}. Required for {monthsstr}'
        else:
            obj = None
            rep = monthsstr
        if obj is None:
            return False, rep

        return True, f"BYMONTH={rep}"

    @classmethod
    def integer(cls, arg, min, max, zero, typ=None):
        """
        :param arg: integer
        :param min: minimum allowed or None
        :param max: maximum allowed or None
        :param zero: zero not allowed if False
        :param typ: label for message
        :return: (True, integer) or (False, message)
        >>> integer(-2, -10, 8, False, 'integer_test')
        (True, -2)
        >>> integer(-2, 0, 8, False, 'integer_test')
        (False, 'integer_test: -2 is less than the allowed minimum')
        """
        msg = ''
        try:
            arg = int(arg)
        except:
            if typ:
                return False, '{}: {}'.format(typ, arg)
            else:
                return False, arg
        if min is not None and arg < min:
            msg = '{} is less than the allowed minimum'.format(arg)
        elif max is not None and arg > max:
            msg = '{} is greater than the allowed maximum'.format(arg)
        elif not zero and arg == 0:
            msg = '0 is not allowed'
        if msg:
            if typ:
                return False, '{}: {}'.format(typ, msg)
            else:
                return False, msg
        else:
            return True, arg

    @classmethod
    def integer_list(cls, arg, min, max, zero, typ=None):
        """
        :param arg: comma separated list of integers
        :param min: minimum allowed or None
        :param max: maximum allowed or None
        :param zero: zero not allowed if False
        :param typ: label for message
        :return: (True, list of integers) or (False, messages)
        >>> integer_list([-13, -10, 0, "2", 27], -12, +20, True, 'integer_list test')
        (False, 'integer_list test: -13 is less than the allowed minimum; 27 is greater than the allowed maximum')
        >>> integer_list([0, 1, 2, 3, 4], 1, 3, True, "integer_list test")
        (False, 'integer_list test: 0 is less than the allowed minimum; 4 is greater than the allowed maximum')
        >>> integer_list("-1, 1, two, 3", None, None, True, "integer_list test")
        (False, 'integer_list test: -1, 1, two, 3')
        >>> integer_list([1, "2", 3], None, None, True, "integer_list test")
        (True, [1, 2, 3])
        """
        if type(arg) == str:
            try:
                args = [int(x) for x in arg.split(',')]
            except:
                if typ:
                    return False, '{}: {}'.format(typ, arg)
                else:
                    return False, arg
        elif type(arg) == list:
            try:
                args = [int(x) for x in arg]
            except:
                if typ:
                    return False, '{}: {}'.format(typ, arg)
                else:
                    return False, arg
        elif type(arg) == int:
            args = [arg]
        msg = []
        ret = []
        for arg in args:
            ok, res = cls.integer(arg, min, max, zero, None)
            if ok:
                ret.append(res)
            else:
                msg.append(res)
        if msg:
            if typ:
                return False, '{}: {}'.format(typ, '; '.join(msg))
            else:
                return False, '; '.join(msg)
        else:
            return True, ret

    @classmethod
    def split_int_str(cls, s):
        match = re.match(r'^([+-]?\d*)(.{1,})$', s)
        if match:
            integer_part = match.group(1)
            string_part = match.group(2)
            # Convert integer_part to an integer if it's not empty, otherwise None
            integer_part = integer_part if integer_part else None
            string_part = string_part if string_part else None
            return integer_part, string_part
        return None, None  # Default case if no match is found

    def do_rdate(self, token):
        # Process rdate token
        print(f"Processing rdate token: {token}")
        parts = token.split()
        try:
            dates = [parse(dt) for dt in parts[1].split(',')]
            return True, dates, []
        except ValueError as e:
            return False, f"Invalid rdate: {parts[1]}. Error: {e}", []

    def do_exdate(self, token):
        # Process exdate token
        print(f"Processing exdate token: {token}")
        parts = token.split()
        try:
            dates = [parse(dt) for dt in parts[1].split(',')]
            return True, dates, []
        except ValueError as e:
            return False, f"Invalid exdate: {parts[1]}. Error: {e}", []

    def finalize_rruleset(self):
        # Finalize the rruleset after collecting all related tokens
        if not self.rrule_tokens:
            return False, "No rrule tokens to process"

        components = []
        rruleset_str = ""
        print(f"finalizing rruleset using {len(self.rrule_tokens) = }; {len(components) = }; {len(rruleset_str) = }")
        for token in self.rrule_tokens:
            rule_parts = []
            _, rrule_params = token
            print(f"finalizing rrule {token = }:  {_ = } with {rrule_params = }")
            dtstart = rrule_params.pop('DTSTART', None)
            if dtstart:
                components.append(f"DTSTART:{dtstart}")
            freq = rrule_params.pop('FREQ', None)
            if freq:
                rule_parts = [f"RRULE:FREQ={freq}",]
            for k, v in rrule_params.items():
                if v:
                    rule_parts.append(f"{v}")

            rule = ";".join(rule_parts)

            components.append(rule)

        for rdate in self.rdates:
            components.append(f"RDATE:{rdate.strftime('%Y%m%dT%H%M%S')}")

        for exdate in self.exdates:
            components.append(f"EXDATE:{exdate.strftime('%Y%m%dT%H%M%S')}")

        rruleset_str = "\n".join(components)
        self.parsed_data['rruleset'] = rruleset_str

        # self.rrule_tokens = []
        self.rdates = []
        self.exdates = []
        return True, rruleset_str

# Example usage
item = Item()
partial_strings = [
    "",
    "- ",
    "- T",
    "- Thanksgiving ",
    "- Thanksgiving @",
    "- Thanksgiving @s 11/26",
    "- Thanksgiving @s 2010/11/26 ",
    "* Thanksgiving @s 2010/11/26 ",
    "* Thanksgiving @s 2010/11/26 @",
    "* Thanksgiving @s 2010/11/26 @r ",
    "* Thanksgiving @s 2010/11/26 @r z",
    "* Thanksgiving @s 2010/11/26 @r y ",
    "* Thanksgiving @s 2010/11/26 @r y &",
    "* Thanksgiving @s 2010/11/26 @r y &M 11 ",
    "* Thanksgiving @s 2010/11/26 @r y &M 11 &w +4",
    "* Thanksgiving @s 2010/11/26 @r y &M 11 &w +4TH",
]

print("\nparsing partial_strings")
for s in partial_strings:
    print(f"\nprocessing: {s}")
    try:
        item.parse_input(s)
        if item.rrule_tokens:
            success, rruleset_str = item.finalize_rruleset()
            print(f"{success = }\n   rruleset_str = {rruleset_str}")
    except Exception as e:
        print(f"   {e = }")

item2 = Item()
item2.parse_input("* multiple rules @s wed 8a @r y &M 11 &w +4TH @r m &w +2TH")
if item2.rrule_tokens:
    success, rruleset_str = item2.finalize_rruleset()
    print(f"{success = }\nrruleset_str:\n{rruleset_str}" )




parsing partial_strings

processing: 
entry to tokens:
   ||
   ||
   []

processing: - 
entry to tokens:
   |12|
   |- |
   [('-', 0, 1)]
Dispatching token: - as itemtype
Processing item type token: -

processing: - T
entry to tokens:
   |123|
   |- T|
   [('-', 0, 1), ('T', 2, 3)]
processing changed token: ('T', 2, 3)
Dispatching token: T as summary
Processing summary token: T

processing: - Thanksgiving 
entry to tokens:
   |123456789012345|
   |- Thanksgiving |
   [('-', 0, 1), ('Thanksgiving ', 2, 15)]
processing changed token: ('Thanksgiving ', 2, 15)
Dispatching token: Thanksgiving  as summary
Processing summary token: Thanksgiving 

processing: - Thanksgiving @
entry to tokens:
   |1234567890123456|
   |- Thanksgiving @|
   [('-', 0, 1), ('Thanksgiving ', 2, 15), ('@', 15, 16)]
processing changed token: ('@', 15, 16)
No handler for token: @

processing: - Thanksgiving @s 11/26
entry to tokens:
   |12345678901234567890123|
   |- Thanksgiving @s 11/26|
   [('-', 0, 1), ('Thanksg

In [4]:
import re

def get_sub_tokens(entry):
    pattern = r'(@\w+ [^&]+)|(^\S+)|(\S[^&]*)'
    matches = re.finditer(pattern, entry)
    tokens_with_positions = []
    for match in matches:
        # Get the matched token
        token = match.group(0)
        # Get the start and end positions
        start_pos = match.start()
        end_pos = match.end()
        # Append the token and its positions as a tuple
        tokens_with_positions.append((token, start_pos, end_pos))
    print(f"{tokens_with_positions = }")

get_sub_tokens("@r y &M 11 &")
get_sub_tokens("@r y &M 11 &w +4TH")
get_sub_tokens("@j cut pieces &i b &p a")


tokens_with_positions = [('@r y ', 0, 5), ('&M 11 ', 5, 11), ('&', 11, 12)]
tokens_with_positions = [('@r y ', 0, 5), ('&M 11 ', 5, 11), ('&w +4TH', 11, 18)]
tokens_with_positions = [('@j cut pieces ', 0, 14), ('&i b ', 14, 19), ('&p a', 19, 23)]


View data "rows" are lists of hashes with keys for *doc_id*, *sort*, *groupby* (YrWk or path) and *columns*. 

- Note that sort must give an order consistent with groupby. E.g., sort by datetime will be consistent with groupby "YrWk".

- Note also that groupby "YrWk" obviates the need for creating a hash with YrWk keys.

As below, the list of hashes is sorted by the "sort" key using itemgetter and then grouped by the "YrWk" or "path" key using groupby. 

In [2]:
import itertools
import operator

data = [
    {'name': 'Alice', 'age': 30},
    {'name': 'Bob', 'age': 30},
    {'name': 'Charlie', 'age': 25},
    {'name': 'David', 'age': 30},
    {'name': 'Eve', 'age': 25}
]

# Sort the data by 'age' using operator.itemgetter
data.sort(key=operator.itemgetter('age'))

# Group by the 'age' key using operator.itemgetter
grouped = itertools.groupby(data, key=operator.itemgetter('age'))

for key, group in grouped:
    print(f"Age: {key}")
    for item in group:
        print(f"  {item}")


Age: 25
  {'name': 'Charlie', 'age': 25}
  {'name': 'Eve', 'age': 25}
Age: 30
  {'name': 'Alice', 'age': 30}
  {'name': 'Bob', 'age': 30}
  {'name': 'David', 'age': 30}


In [16]:
import re
from datetime import datetime

class Item:
    freq_map = dict(
        y='YEARLY', m='MONTHLY', w='WEEKLY', d='DAILY', h='HOURLY', n='MINUTELY'
    )

    key_to_param = dict(
        i='INTERVAL', c='COUNT', s='BYSETPOS', u='UNTIL', M='BYMONTH', m='BYMONTHDAY',
        W='BYWEEKNO', w='BYDAY', h='BYHOUR', n='BYMINUTE', E='BYEASTER'
    )

    param_to_key = {v: k for k, v in key_to_param.items()}

    def rrule_to_entry(self, rstr: str)->str:
        # Split the input text into lines
        lines = rstr.strip().split('\n')

        dtstart_list = []
        rrule_list = []
        rdate_list = []
        exdate_list = []

        for line in lines:
            if line.startswith("DTSTART:"):
                dtstart_str = line.replace("DTSTART:", "")
                dtstart_list.append(dtstart_str)
            elif line.startswith("RRULE:"):
                rrule_str = line.replace("RRULE:", "")
                rrule_list.append(rrule_str)
            elif line.startswith("RDATE:"):
                rdate_str = line.replace("RDATE:", "")
                rdate_list.extend(rdate_str.split(',')) # Split multiple RDATEs
            elif line.startswith("EXDATE:"):
                exdate_str = line.replace("EXDATE:", "")
                exdate_list.extend(exdate_str.split(',')) # Split multiple EXDATEs

        # Process DTSTART
        dtstart_part = ""
        if dtstart_list:
            dtstart_date = datetime.strptime(dtstart_list[0], "%Y%m%dT%H%M%S")
            dtstart_part = f"@s {dtstart_date.strftime('%Y-%m-%d %-I:%M%p').lower()}"

        # Process RRULEs
        rrule_parts = []
        for rrule_str in rrule_list:
            rrule_params = {}
            for param in rrule_str.split(";"):
                key, value = param.split("=")
                rrule_params[key] = value

            freq_entry = list(self.freq_map.keys())[list(self.freq_map.values()).index(rrule_params['FREQ'])]
            rrule_part = f"@r {freq_entry}"

            for key, value in rrule_params.items():
                if key == 'FREQ':
                    continue
                entry = self.param_to_key[key]
                rrule_part += f" &{entry} {value}"

            rrule_parts.append(rrule_part)

        # Process RDATEs
        rdate_parts = []
        for rdate_str in rdate_list:
            rdate_date = datetime.strptime(rdate_str, "%Y%m%dT%H%M%S")
            rdate_parts.append(f"@+ {rdate_date.strftime('%Y-%m-%d %-I:%M%p').lower()}")

        # Process EXDATEs
        exdate_parts = []
        for exdate_str in exdate_list:
            exdate_date = datetime.strptime(exdate_str, "%Y%m%dT%H%M%S")
            exdate_parts.append(f"@- {exdate_date.strftime('%Y-%m-%d %-I:%M%p').lower()}")

        return f"{dtstart_part} {' '.join(rrule_parts)} {' '.join(rdate_parts)} {' '.join(exdate_parts)}"
        # return f"{' '.join(rrule_parts)} {' '.join(rdate_parts)} {' '.join(exdate_parts)}"

# Example usage
item = Item()
input_str = "DTSTART:20240807T160000\nRRULE:FREQ=WEEKLY;INTERVAL=2;BYDAY=WE\nDTSTART:20240807T160000\nRRULE:FREQ=WEEKLY;BYDAY=MO\nRDATE:20240809T144500,20240816T143000\nEXDATE:20240821T160000"
output_str = item.rrule_to_entry(input_str)

print(output_str)


@s 2024-08-07 4:00pm @r w &i 2 &w WE @r w &w MO @+ 2024-08-09 2:45pm @+ 2024-08-16 2:30pm @- 2024-08-21 4:00pm


In [18]:
from ZODB import DB, FileStorage
from persistent import Persistent
import transaction

# Define a simple persistent class
class Task(Persistent):
    def __init__(self, title, completed=False):
        self.title = title
        self.completed = completed

    def mark_completed(self):
        self.completed = True

# Create a ZODB database stored in a file
storage = FileStorage.FileStorage('tasks.fs')
db = DB(storage)

# Open a connection to the database
connection = db.open()

# Get the root object
root = connection.root()

# Add a new task to the database if it doesn't exist
if 'tasks' not in root:
    root['tasks'] = []

# Create a new task and add it to the tasks list
task = Task(title="Buy groceries")
root['tasks'].append(task)

# Commit the transaction to save the changes
transaction.commit()

# Retrieve and display tasks from the database
for task in root['tasks']:
    print(f"Task: {task.title}, Completed: {task.completed}")

# Mark the first task as completed and save
root['tasks'][0].mark_completed()
transaction.commit()

# Display tasks again after marking the first as completed
for task in root['tasks']:
    print(f"Task: {task.title}, Completed: {task.completed}")

# Close the connection and the database
connection.close()
db.close()


Task: Buy groceries, Completed: False
Task: Buy groceries, Completed: True
